In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
tqdm.pandas()

/root/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

print('gen_prev_ans_ques_set...')

gen_prev_ans_ques_set...


In [3]:
data = pd.read_pickle(os.path.join(DATA_PATH, 'invite_data.pkl'))
prev_ans_ques = pd.read_pickle(os.path.join(DATA_PATH, 'prev_ans_ques.pkl'))
question_info = pd.read_pickle(os.path.join(DATA_PATH, 'question_info.pkl'))

In [4]:
data = pd.concat([data, prev_ans_ques], axis=1)

In [5]:
data['prev_ans_ques'] = data['prev_ans_ques'].progress_apply(lambda x: x if type(x) == list else [])

100%|██████████| 10630845/10630845 [00:16<00:00, 642395.54it/s]


In [6]:
data['ques1'] = data['prev_ans_ques'].progress_apply(lambda x: x[-1] if len(x) >= 1 else np.nan)
data['ques2'] = data['prev_ans_ques'].progress_apply(lambda x: x[-2] if len(x) >= 2 else np.nan)
data['ques3'] = data['prev_ans_ques'].progress_apply(lambda x: x[-3] if len(x) >= 3 else np.nan)

100%|██████████| 10630845/10630845 [00:14<00:00, 757157.60it/s]


In [7]:
question_info = question_info[['qid', 'title_w', 'desc_w']]
data = data.merge(question_info, 'left', 'qid')

In [8]:
question_info.columns = ['ques1', 'title_w1', 'desc_w1']
data = data.merge(question_info, 'left', 'ques1')

In [9]:
question_info.columns = ['ques2', 'title_w2', 'desc_w2']
data = data.merge(question_info, 'left', 'ques2')

In [10]:
question_info.columns = ['ques3', 'title_w3', 'desc_w3']
data = data.merge(question_info, 'left', 'ques3')

In [11]:
for c in ['title_w1', 'desc_w1', 'title_w2', 'desc_w2', 'title_w3', 'desc_w3']:
    data[c] = data[c].progress_apply(lambda x: [] if type(x) == float else x)

100%|██████████| 10630845/10630845 [00:15<00:00, 702616.84it/s]


In [12]:
for c in ['desc_w1', 'desc_w2', 'desc_w3']:
    data['len_' + c] = data[c].progress_apply(len)

100%|██████████| 10630845/10630845 [00:11<00:00, 957672.31it/s] 


In [13]:
data['len_desc_q'] = data['len_desc_w1'] + data['len_desc_w2'] + data['len_desc_w3']
data['len_q'] = data['prev_ans_ques'].apply(len)

In [14]:
(data['len_desc_q'] > 256).sum()

220814

In [15]:
data['title_q'] = data['title_w']
data['desc_q'] = data['desc_w']

In [16]:
data['title_u'] = data['title_w1'] + data['title_w2'] + data['title_w3']
print(data['title_u'].apply(len).max())

64


In [17]:
def func(row):
    if row['len_desc_q'] > 256:
        return row['desc_w1'][:85] + row['desc_w2'][:85] + row['desc_w3'][:85]
    else:
        return row['desc_w1'] + row['desc_w2'] + row['desc_w3']

data['desc_u'] = data.progress_apply(func, axis=1)

100%|██████████| 10630845/10630845 [10:04<00:00, 17579.57it/s]


In [18]:
print(data['desc_u'].apply(len).max())

256


In [19]:
for c in ['title_q', 'desc_q', 'title_u', 'desc_u']:
    data[c] = data[c].progress_apply(lambda x: [0] if len(x) == 0 else x)

100%|██████████| 10630845/10630845 [00:26<00:00, 407844.26it/s] 


In [20]:
data.head()

,iday,ihour,itime,label,qid,uid,prev_ans_ques,ques1,ques2,ques3,...,desc_w3,len_desc_w1,len_desc_w2,len_desc_w3,len_desc_q,len_q,title_q,desc_q,title_u,desc_u
0,3865,22,92782,0.0,2166419046,401693808,[],NaN,NaN,NaN,...,[],0,0,0,0,0,"[11058, 272, 2202, 431, 951, 243, 3828, 3037, ...","[3828, 14870, 3037, 2450, 692, 1688, 53547, 24...",[0],[0]
1,3844,11,92267,0.0,1550017551,3392373099,[],NaN,NaN,NaN,...,[],0,0,0,0,0,"[149, 79, 5210, 22869]","[429, 2709, 1465, 6965]",[0],[0]
2,3862,15,92703,0.0,604029601,2317670257,"[3034061280, 1309562090]",1.309562e+09,3.034061e+09,NaN,...,[],1,38,0,39,2,"[31489, 9218, 3440, 243]","[3046, 221, 3440, 730, 31489, 9218, 1278, 415]","[1116, 6761, 17922, 272, 263, 4534, 7742]","[0, 272, 4534, 7742, 3738, 1277, 111, 2425, 41..."
3,3849,11,92387,0.0,2350061229,1618461867,[],NaN,NaN,NaN,...,[],0,0,0,0,0,"[973, 64958, 8583, 2269, 628, 565, 2200, 590, ...",[0],[0],[0]
4,3867,4,92812,0.0,2443223942,3544409350,[],NaN,NaN,NaN,...,[],0,0,0,0,0,"[554, 28208, 396, 51]",[0],[0],[0]


In [21]:
data[['title_q', 'desc_q', 'title_u', 'desc_u']].to_pickle(os.path.join(DATA_PATH, 'prev_ans_ques_title_desc.pkl'))

In [22]:
toc = time.time()
print('Used time: %d' % int(toc-tic))

Used time: 1029


In [23]:
data['id'] = data['qid'].apply(str) + '_' + data['uid'].apply(str)

In [24]:
tmp = data['id'].value_counts() 
tmp[tmp > 1]

3766912814_975408302     12
1656708750_1891662041     8
1656708750_185871998      7
2730406015_1088150594     7
374366132_1041483084      6
                         ..
211639478_3168997882      2
2982884061_3672634699     2
1890057286_2377901070     2
1542924553_2490140348     2
3012588586_2302975115     2
Name: id, Length: 2336, dtype: int64

In [25]:
tmp2 = data[data['id'].isin(tmp[tmp > 1].index)][['id', 'label']]
tmp2

,id,label
60,2447267784_3941659382,0.0
1189,563440782_2658847573,1.0
2960,3475759373_1473282171,1.0
3004,2346951238_460869274,1.0
4511,1664816545_552057782,1.0
...,...,...
10621215,890298799_4008185451,NaN
10621918,1446146454_2821601124,NaN
10625670,3977768775_3981602523,NaN
10627984,1818189446_3246358942,NaN


In [26]:
tmp3 = tmp2[pd.isna(tmp2['label'])]
tmp4 = tmp2[~pd.isna(tmp2['label'])]

In [27]:
tmp5 = tmp3[tmp3['id'].isin(tmp4['id'])]
tmp5

,id,label
9501254,3349856436_1662430319,NaN
9505965,3977768775_174458543,NaN
9520429,495966293_135201613,NaN
9522517,696541928_1230378210,NaN
9525636,13601293_356313381,NaN
...,...,...
10599431,1791424099_1604514588,NaN
10611680,1828489264_2793566886,NaN
10618097,2538637960_1751107848,NaN
10621918,1446146454_2821601124,NaN


In [28]:
tmp6 = tmp4[tmp4['id'].isin(tmp3['id'])]
tmp6

,id,label
36212,857378709_2536436801,1.0
173505,843902755_529222996,0.0
201174,2381892828_3168997882,0.0
204348,4183467948_474370804,0.0
221098,495966293_135201613,0.0
...,...,...
9021654,3404694841_524492604,0.0
9106194,510864453_1295500617,1.0
9188595,2153395118_3053602909,0.0
9318568,1729091216_167877392,0.0


In [29]:
tmp6['id'].nunique()

119

In [30]:
tmp7 = tmp6.groupby('id')['label'].apply(lambda x: list(x)).reset_index()
tmp7['len'] = tmp7['label'].apply(len)
tmp7

,id,label,len
0,1162178181_336055354,[1.0],1
1,1195238336_3803870696,[1.0],1
2,13601293_356313381,[1.0],1
3,1372715791_1956070876,[0.0],1
4,1375884537_4126196603,[1.0],1
...,...,...,...
114,893889544_1777561150,[0.0],1
115,893889544_3678601289,[0.0],1
116,924722639_4221144503,[0.0],1
117,966303606_1477594286,[0.0],1


In [31]:
tmp7[tmp7['len'] > 1]

,id,label,len
6,1430577052_2435805188,"[1.0, 1.0, 1.0, 1.0]",4
7,1430577052_3594373145,"[1.0, 1.0, 1.0, 1.0, 1.0]",5
73,3712599104_2678718278,"[1.0, 1.0]",2
76,3766912814_4175838558,"[0.0, 0.0]",2
77,3789567229_4270234802,"[1.0, 1.0]",2
78,3797510006_2712312266,"[1.0, 1.0, 1.0]",3
79,3802123318_2037494776,"[0.0, 0.0]",2
98,4183467948_903249500,"[0.0, 0.0]",2
105,571680622_2976534255,"[1.0, 1.0]",2


In [32]:
tmp7['post_label'] = tmp7['label'].apply(lambda x: x[0])
tmp7

,id,label,len,post_label
0,1162178181_336055354,[1.0],1,1.0
1,1195238336_3803870696,[1.0],1,1.0
2,13601293_356313381,[1.0],1,1.0
3,1372715791_1956070876,[0.0],1,0.0
4,1375884537_4126196603,[1.0],1,1.0
...,...,...,...,...
114,893889544_1777561150,[0.0],1,0.0
115,893889544_3678601289,[0.0],1,0.0
116,924722639_4221144503,[0.0],1,0.0
117,966303606_1477594286,[0.0],1,0.0


In [33]:
tmp8 = tmp7[['id', 'post_label']]
tmp8

,id,post_label
0,1162178181_336055354,1.0
1,1195238336_3803870696,1.0
2,13601293_356313381,1.0
3,1372715791_1956070876,0.0
4,1375884537_4126196603,1.0
...,...,...
114,893889544_1777561150,0.0
115,893889544_3678601289,0.0
116,924722639_4221144503,0.0
117,966303606_1477594286,0.0


In [34]:
tmp8.to_pickle(os.path.join(DATA_PATH, 'post_label.pkl'))